## Graph

In [61]:
from graphviz import Graph
from collections import defaultdict

class Graph:
    def __init__(self, edges, directed=False):
        self.directed = directed
        self._vertices = set()
        self.num_vertices = 0
        self.num_edges = 0
        self.edges = defaultdict(list)
        for edge in edges:
            self.insert_edge(edge)
            
    def insert_edge(self, edge):
        u, v, w = edge
        self._vertices.add(u)
        self._vertices.add(v)
        self.edges[u].append((v, w))
        if not self.directed:
            self.edges[v].append((u, w))
        self.num_vertices = len(self._vertices)
        
    def visualize(self):
        graph = Graph(engine="sfdp")
        for p_vertex in self.edges:
            graph.node(str(p_vertex))
            for (c_vertex, weight) in self.edges[p_vertex]:
                graph.node(str(c_vertex))
                graph.edge(str(p_vertex), str(c_vertex), label=str(weight))
        
        return graph
        
    def __str__(self):
        out_str = ""
        for p_vertex, children in self.edges.items():
            p_str = "{} :".format(p_vertex)
            for c_vertex, weight in children:
                p_str += "({}, {}) ".format(c_vertex, weight)
            out_str += p_str + "\n"
        
        return out_str

## Prim's Algorithm

In [62]:
import heapq as hq

def prim(graph, start=None):
    if start is None:
        start = next(iter(graph.edges))
    visited = [False] * len(graph.edges)
    distances = [(0, start)]
    total_weight = 0
    while distances:
        weight, vertex = hq.heappop(distances)
        if not visited[vertex]:
            visited[vertex] = True
            total_weight += weight
            for vertex, weight in graph.edges[vertex]:
                hq.heappush(distances, (weight, vertex))
    return total_weight

In [63]:
g = Graph(
    [(0, 1, 5), (0, 2, 7), (0, 3, 12), (1, 4, 7), (4, 5, 2), (1, 2, 9), (3, 2, 4), (4, 2, 4), (3, 5, 7), (5, 2, 3), (5, 6, 2), (4, 6, 5)]
)

In [64]:
minimum_total_weight = prim(g, start=0)

In [65]:
minimum_total_weight

23

## Kruskal's Algorithm

In [84]:
class UnionFind:
    def __init__(self, nodes):
        self.nodes = nodes
        self.num_components = len(self.nodes)
        self.sizes = [1] * len(self.nodes)
        
    def find(self, x):
        parent = self.nodes[x]
        while parent != self.nodes[parent]:
            parent = self.nodes[parent]
        self.nodes[x] = parent
        
        return parent
    
    def union(self, x, y):
        p_x, p_y = self.find(x), self.find(y)
        if p_x != p_y:
            if self.sizes[p_x] < self.sizes[p_y]:
                self.nodes[p_x] = p_y
                self.sizes[p_y] += self.sizes[p_x]
                self.num_components -= 1
            else:
                self.nodes[p_y] = p_x
                self.sizes[p_x] += self.sizes[p_y]
                self.num_components -= 1

In [108]:
def kruskal(graph):
    total_cost = 0
    MST_edges = []
    edges = []
    union_find = UnionFind(list(range(len(graph.edges))))
    for p_vertex in graph.edges:
        for (c_vertex, weight) in graph.edges[p_vertex]:
            hq.heappush(edges, (weight, p_vertex, c_vertex))
    while edges:
        weight, p_vertex, c_vertex = hq.heappop(edges)
        if union_find.find(p_vertex) != union_find.find(c_vertex):
            MST_edges.append((p_vertex, c_vertex))
            total_cost += weight
            union_find.union(p_vertex, c_vertex)
    
    return total_cost, MST_edges

In [109]:
g = Graph(
    [(0, 1, 5), (0, 2, 7), (0, 3, 12), (1, 4, 7), (4, 5, 2), (1, 2, 9), (3, 2, 4), (4, 2, 4), (3, 5, 7), (5, 2, 3), (5, 6, 2), (4, 6, 5)]
)

In [110]:
minimum_total_weight, MST_edges = kruskal(g)

In [111]:
minimum_total_weight

23

In [112]:
MST_edges

[(4, 5), (5, 6), (2, 5), (2, 3), (0, 1), (0, 2)]

## Dijkstra's Algorithm

In [ ]:
import sys

def dijkstra(graph, start=None):
    if start is None:
        start = next(iter(graph.edges))
    edges = [(0, start)]
    visited = []
    distances {i: [sys.maxsize, None] for i in range(len(graph.edges))}
    while edges:
        weight, vertex = hq.heappop(visited)
        if vertex not in visited:
            

In [ ]:
g = Graph(
    [(0, 1, 5), (0, 2, 7), (0, 3, 12), (1, 4, 7), (4, 5, 2), (1, 2, 9), (3, 2, 4), (4, 2, 4), (3, 5, 7), (5, 2, 3), (5, 6, 2), (4, 6, 5)]
)

## Floyd-Warshall Algorithm